In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import pathlib as pl
import pickle
import xarray as xa

In [ ]:
nc_path = pl.Path("../synthetic-valley/data/synthetic_valley_truth.nc")
nc_ds = xa.open_dataset(nc_path)

## Load the existing base model

In [ ]:
name = "sv"
ws = pl.Path("../models/synthetic-valley-base")

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=ws)

In [ ]:
nper = sim.tdis.nper.array
gwf = sim.get_model(name)
nlay, nrow, ncol = gwf.dis.nlay.array, gwf.dis.nrow.array, gwf.dis.ncol.array
botm = gwf.dis.botm.array

### Change the simulation workspace

In [ ]:
ws = ws.parent / "synthetic-valley-advanced"
print(ws)

In [ ]:
sim.set_sim_path(ws)

## Convert the well file to MAW 

In [ ]:
well_data = {
    "reilly": (3, 4, 5, 14, -67000),
    "virginia_city": (3, 4, 34, 15, -268000),
    "virginia_city_alt": (3, 4, 32, 5, -268000),
}

In [ ]:
# <ifno> <radius> <bottom> <strt> <condeqn> <ngwfnodes> [<aux(naux)>] [<boundname>]
package_data = []
for idx, (key, value) in enumerate(well_data.items()):
    k0, k1, i, j, Q = value
    package_data.append((idx, 0.5, float(botm[k1, i, j]), 0.0, "theim", 2, key))
package_data

In [ ]:
# <ifno> <icon> <cellid> <scrn_top> <scrn_bot> <hk_skin> <radius_skin>
connection_data = []
for idx, (key, value) in enumerate(well_data.items()):
    k0, k1, i, j, well_name = value
    for jdx, k in enumerate(range(k0, k1 + 1)):
        cellid = (k, i, j)
        top = float(botm[k - 1, i, j])
        bot = float(botm[k, i, j])
        connection_data.append((idx, jdx, (k, i, j), top, bot, -999.0, -999.0))

connection_data

In [ ]:
maw_spd = {}
for n in range(nper):
    spd = gwf.wel.stress_period_data.get_data(n)
    if spd is not None:
        well_names = spd["boundname"].tolist()
        inactive_wells = [
            well_name for well_name in well_data.keys() if well_name not in well_names
        ]
        active_wells = [
            well_name for well_name in well_data.keys() if well_name in well_names
        ]
    else:
        inactive_wells = list(well_data.keys())
        active_wells = []
    print(active_wells, inactive_wells)
    t = []
    for idx, well_name in enumerate(well_data.keys()):
        if well_name in inactive_wells:
            t.append((idx, "status", "inactive"))
        elif well_name in active_wells:
            k0, k1, i, j, Q = well_data[well_name]
            t.append((idx, "status", "active"))
            t.append((idx, "rate", Q))
    maw_spd[n] = t

maw_spd

In [ ]:
gwf.remove_package("pwell")

In [ ]:
maw = flopy.mf6.ModflowGwfmaw(
    gwf,
    boundnames=True,
    nmawwells=len(well_data),
    packagedata=package_data,
    connectiondata=connection_data,
    perioddata=maw_spd,
    pname="pwell",
)

## Convert RIV package to SFR package

## Convert high-K lake to LAK package

In [ ]:
lake_location = nc_ds["lake_location"].to_numpy()

## Add UZF package

## Add Mover

## Write and run simulation

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

## Plot the results

### Model Properties

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Hydraulic conductivity")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(gwf.npf.k.array[idx], masked_values=[2000000.0])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Bottom Elevation")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(gwf.dis.botm.array[idx])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Cell thickness")
    z = gwf.modelgrid.cell_thickness

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(z[idx])
        ax.set_title(f"Layer {idx + 1}")

### Simulated Heads and Drawdown

In [ ]:
levels = np.arange(2, 20.0, 2)

#### Calibration

In [ ]:
hds = gwf.output.head().get_data(totim=1.0)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - calibration")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=1.0)[0]["q"]
print(f"River infiltration: {np.all(v > 0)}\n{v}")

##### Calculate the residuals

In [ ]:
obs_path = pl.Path("../synthetic-valley/data")
with open(obs_path / "obs_data.pkl", "rb") as f:
    obs_rc_locs, wt_obs, aq_obs, aq_layer = pickle.load(f)

cal_loc_wt = [(0, i, j) for i, j in obs_rc_locs]
cal_loc_aq = [(aq_layer[idx], i, j) for idx, (i, j) in enumerate(obs_rc_locs)]

In [ ]:
sim_wt = np.array([hds[idx] for idx in cal_loc_wt])

In [ ]:
resid_wt = sim_wt - np.array(wt_obs)
resid_wt

In [ ]:
sim_aq = np.array([hds[idx] for idx in cal_loc_aq])

In [ ]:
resid_aq = sim_wt - np.array(aq_obs)
resid_aq

In [ ]:
resid_gb = np.concatenate((resid_wt, resid_aq))

In [ ]:
print(
    f"Water Table Statistics\nMean Error: {resid_wt.mean()} ft.\nRMSE:       {np.sqrt((resid_wt**2).sum()) / resid_wt.shape[0]} ft."
)

In [ ]:
print(
    f"Lower Aquifer Statistics\nMean Error: {resid_aq.mean()} ft.\nRMSE:       {np.sqrt((resid_aq**2).sum()) / resid_aq.shape[0]} ft."
)

In [ ]:
print(
    f"Global Statistics\nMean Error: {resid_gb.mean()} ft.\nRMSE:       {np.sqrt((resid_gb**2).sum()) / resid_gb.shape[0]} ft."
)

##### Plot the residuals

In [ ]:
xy = [
    (float(gwf.modelgrid.xcellcenters[i, j]), float(gwf.modelgrid.ycellcenters[i, j]))
    for i, j in obs_rc_locs
]

In [ ]:
x, y = np.array(xy)[:, 0], np.array(xy)[:, 1]

In [ ]:
grid_x, grid_y = np.meshgrid(gwf.modelgrid.xycenters[0], gwf.modelgrid.xycenters[1])

In [ ]:
# Linearly interpolate the data (x, y) on a grid defined by (xi, yi).
triang = tri.Triangulation(x, y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_wt)
grid_resid_wt = interpolator(grid_x, grid_y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_aq)
grid_resid_aq = interpolator(grid_x, grid_y)

In [ ]:
resid_levels = np.arange(-2, 2.25, 0.25)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 2, figsize=(8, 5), sharey=True)
    fig.suptitle("Residuals")

    ax = axs[0]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_array(lake_location, cmap="Blues_r", masked_values=[0])
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_wt):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]))
    cs = ax.contour(
        grid_x,
        grid_y,
        grid_resid_wt,
        levels=resid_levels,
        linewidths=0.75,
        colors="red",
    )
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Water Table")

    ax = axs[1]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_aq):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]), clip_on=False)
    cs = ax.contour(
        grid_x,
        grid_y,
        grid_resid_aq,
        levels=resid_levels,
        linewidths=0.75,
        colors="red",
    )
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Lower Aquifer")

**NOTE:** There is spatial bias in the simulated results (*i.e.*, residuals are positive in the Northeast and negative in the Southwest).

#### Case A

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Case A")
    hds = gwf.output.head().get_data(totim=2.0)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Case A")
    ddn = gwf.output.head().get_data(totim=1.0) - gwf.output.head().get_data(totim=2.0)

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=2.0)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

 #### Case B

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Case B")
    hds = gwf.output.head().get_data(totim=3.0)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Case B")
    ddn = gwf.output.head().get_data(totim=1.0) - gwf.output.head().get_data(totim=3.0)

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=3.0)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

#### Extra Run

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Extra Run")
    hds = gwf.output.head().get_data(totim=4.0)

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Extra Run")
    ddn = gwf.output.head().get_data(totim=1.0) - gwf.output.head().get_data(totim=4.0)

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=4.0)[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

### Streamflow results

In [ ]:
df = gwf.riv.output.obs().get_dataframe()
df["PF"] /= -86400
df["RIVFLOW"] /= -86400
df["TOTAL"] = df["PF"] + df["RIVFLOW"]
df

In [ ]:
Q = df["TOTAL"].values
Q

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Southern Boundary - Gage 1")

    ax = axs[0]
    ax.set_ylim(5, 12)
    ax.plot(df["totim"], Q, ls="-", marker="o", clip_on=False)
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(0, 50)
    ax.plot(df["totim"], -100.0 * (Q - Q[0]) / Q[0], ls="-", marker="o", clip_on=False)
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")
    ax.set_xticks([1, 2, 3, 4], ["Calibration", "Case A", "Case B", "Extra Run"])

In [ ]:
Q = df["PF"].values
Q

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Pollock's Ford - Gage 2")

    ax = axs[0]
    ax.set_ylim(1, 1.6)
    ax.plot(df["totim"], Q, ls="-", marker="o", clip_on=False)
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(0, 50)
    ax.plot(df["totim"], -100.0 * (Q - Q[0]) / Q[0], ls="-", marker="o", clip_on=False)
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")
    ax.set_xticks([1, 2, 3, 4], ["Calibration", "Case A", "Case B", "Extra Run"])

### Lake stage

In [ ]:
fpth = ws / f"{name}.gwf.scenario.csv"

In [ ]:
df = flopy.utils.Mf6Obs(fpth).get_dataframe()
df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(1, 1, figsize=(9, 1.5))

    ax.plot(
        df["totim"],
        df["LAKE-N"],
        ls="-",
        marker="o",
    )
    ax.set_ylabel("Lake\nStage, ft")
    ax.set_xlabel("Stress Period")
    ax.set_xticks([1, 2, 3, 4], ["Calibration", "Case A", "Case B", "Extra Run"])
    ax.set_ylim(8, 12)